<a href="https://colab.research.google.com/github/SeanMuInCa/learn_python/blob/master/groupassignment2025retry2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Data Preprocessing

import pandas as pd
import numpy as np

# Load the dataset (ensure 'train.csv' is in the working directory)
df = pd.read_csv('train.csv')

# Print the initial shape of the dataset
print("Initial shape of train data:", df.shape)

# Drop any rows with missing values
df = df.dropna()
print("Shape after dropping missing values:", df.shape)

# Check if any force_meas values are non-positive
if (df['force_meas'] <= 0).any():
    raise ValueError("There are non-positive values in force_meas!")
else:
    print("All force_meas values are positive.")

# Display basic information and the first 5 rows of the dataset
print("Dataset info:")
print(df.info())
print("First 5 rows:")
print(df.head())


Initial shape of train data: (14994, 35)
Shape after dropping missing values: (14993, 35)
All force_meas values are positive.
Dataset info:
<class 'pandas.core.frame.DataFrame'>
Index: 14993 entries, 0 to 14992
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tappingsteelgrade     14993 non-null  object 
 1   force_meas            14993 non-null  float64
 2   speed                 14993 non-null  float64
 3   entrytemperature      14993 non-null  float64
 4   entrytemperaturebot   14993 non-null  float64
 5   entrytemperaturecore  14993 non-null  float64
 6   entrytemperaturetop   14993 non-null  float64
 7   entrythickness        14993 non-null  float64
 8   entrywidth            14993 non-null  float64
 9   exitthickness         14993 non-null  float64
 10  zeropoint             14993 non-null  float64
 11  radius                14993 non-null  float64
 12  pctal                 14993 non-null 

In [2]:
# Select features by dropping the target column 'force_meas'
features = df.drop(columns=['force_meas'])
target = df['force_meas']

# Convert the categorical column 'tappingsteelgrade' using one-hot encoding
if 'tappingsteelgrade' in features.columns:
    features = pd.get_dummies(features, columns=['tappingsteelgrade'], drop_first=True)

print("Features shape after encoding:", features.shape)


from sklearn.preprocessing import StandardScaler

# Scale the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

print("Feature scaling completed. Sample of scaled features:")
print(X_scaled[:5])


Features shape after encoding: (14993, 56)
Feature scaling completed. Sample of scaled features:
[[-1.20815045e+00  1.62317851e+00  1.10172575e+00  1.76238659e+00
   9.87948550e-01  1.45623695e+00  3.61472968e-01  1.28464984e+00
  -2.83133583e-01  1.73464961e+00 -8.51661554e-01 -1.95072152e+00
   1.28237237e+00  2.64962246e-01  0.00000000e+00 -4.82318706e-01
  -3.95594767e-01  3.58462510e-01  0.00000000e+00 -2.04614134e+00
  -8.06154473e-01 -8.24254455e-01 -1.19837786e+00 -3.61909645e-01
  -1.46499377e+00  1.43656069e+00  1.66689327e+00 -1.48847140e+00
  -7.13871890e-01 -9.52200096e-01 -1.07879267e+00 -1.65922425e+00
   4.48053053e-01 -4.08684453e-02  3.14958465e+00 -3.00138110e-01
  -6.74989919e-02 -1.63422278e-01 -3.16459922e-02 -1.63359226e-02
  -1.25915578e-01 -1.50482932e-01 -3.93355942e-01 -3.45032780e-01
  -3.10709877e-01 -2.82072849e-01 -2.28342617e-01 -3.46699198e-02
  -1.09612270e-01 -1.63359226e-02 -3.65477450e-02 -1.15504710e-02
  -2.73717255e-01 -1.82647259e-02 -3.97865908

In [3]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Split the scaled features and target into training and validation sets
# 注意：features_scaled 是经过之前预处理与特征缩放后的数据（形状为 (18740, 56)），
# y 是目标变量 force_meas
X_train, X_val, y_train, y_val = train_test_split(X_scaled, df['force_meas'], test_size=0.2, random_state=42)

# Define 5-fold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)


lr = LinearRegression()
lr_cv_scores = cross_val_score(lr, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv)
print("Linear Regression CV MAE: {:.4f}".format(-np.mean(lr_cv_scores)))

# -------------------------------
# Decision Tree Regressor with Grid Search
# -------------------------------
dt = DecisionTreeRegressor(random_state=42)
dt_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
dt_grid = GridSearchCV(dt, dt_param_grid, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
dt_grid.fit(X_train, y_train)
print("Decision Tree best params:", dt_grid.best_params_)
print("Decision Tree CV MAE: {:.4f}".format(-dt_grid.best_score_))

# -------------------------------
# Random Forest Regressor with Grid Search
# -------------------------------
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
rf_grid = GridSearchCV(rf, rf_param_grid, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
rf_grid.fit(X_train, y_train)
print("Random Forest best params:", rf_grid.best_params_)
print("Random Forest CV MAE: {:.4f}".format(-rf_grid.best_score_))

# -------------------------------
# Gradient Boosting Regressor with Grid Search
# -------------------------------
gb = GradientBoostingRegressor(random_state=42)
gb_param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5]
}
gb_grid = GridSearchCV(gb, gb_param_grid, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
gb_grid.fit(X_train, y_train)
print("Gradient Boosting best params:", gb_grid.best_params_)
print("Gradient Boosting CV MAE: {:.4f}".format(-gb_grid.best_score_))

# -------------------------------
# Evaluate each model on the validation set
# -------------------------------
models = {
    'Linear Regression': lr,
    'Decision Tree': dt_grid.best_estimator_,
    'Random Forest': rf_grid.best_estimator_,
    'Gradient Boosting': gb_grid.best_estimator_
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"{name} Validation MAE: {mae:.4f}")
    print(f"{name} Validation RMSE: {rmse:.4f}")


Linear Regression CV MAE: 5565531.4078
Decision Tree best params: {'max_depth': 10, 'min_samples_split': 10}
Decision Tree CV MAE: 6599853.9875
Random Forest best params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Random Forest CV MAE: 5252093.7735
Gradient Boosting best params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Gradient Boosting CV MAE: 4989130.7414
Linear Regression Validation MAE: 5567526.7685
Linear Regression Validation RMSE: 7214382.6856
Decision Tree Validation MAE: 6520268.7094
Decision Tree Validation RMSE: 8655189.8080
Random Forest Validation MAE: 5127791.6150
Random Forest Validation RMSE: 6684445.4882
Gradient Boosting Validation MAE: 5006398.8995
Gradient Boosting Validation RMSE: 6425564.8506


In [8]:
# Save the fitted scaler for later use
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)  # Save the scaler object

# Save the best Random Forest model (obtained from GridSearchCV)
with open('best_rf_model.pkl', 'wb') as f:
    pickle.dump(rf_grid.best_estimator_, f)  # Save the best RF model

# Save the best Gradient Boosting model (obtained from GridSearchCV)
with open('best_gb_model.pkl', 'wb') as f:
    pickle.dump(gb_grid.best_estimator_, f)  # Save the best GB model

# Save the training columns (features names) for later use
training_columns = features.columns.tolist()  # 'features' 是经过预处理后用于训练的 DataFrame
with open('training_columns.pkl', 'wb') as f:
    pickle.dump(training_columns, f)
print("Training columns have been saved successfully.")

Training columns have been saved successfully.


In [11]:
# ===============================
# Step 11: Final Evaluation on Test Data
# ===============================

import pandas as pd
import numpy as np
import pickle
import time
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load test data (ensure test.csv is in your working directory)
test_df = pd.read_csv('test.csv')

# Drop missing values if any (you may adjust based on your preprocessing strategy)
test_df = test_df.dropna()

# For test data, note: do not use the 'force_pre' column for prediction, it's the baseline.
# We assume test_df contains 'force_pre' and other features including 'tappingsteelgrade'
# Also assume that 'force_meas' is present for evaluation
# If not, adjust accordingly (for final evaluation, force_meas is needed)

# Separate baseline predictions for later comparison
baseline_force_pre = test_df['force_pre']

# Separate target variable for evaluation (force_meas)
y_test = test_df['force_meas']

# ===============================
# Preprocessing: Feature Selection & Encoding
# ===============================
# Exclude the target 'force_meas' and baseline column 'force_pre'
features_test = test_df.drop(columns=['force_meas', 'force_pre'])

# One-hot encode the categorical column 'tappingsteelgrade' (using the same columns as during training)
features_test = pd.get_dummies(features_test, columns=['tappingsteelgrade'], drop_first=True)

# IMPORTANT: Align the test features with the training features.
# Load the fitted scaler (assumes scaler.pkl was saved during training)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# To ensure that test data has the same feature columns as training data,
# we reindex the test DataFrame to match training columns.
# Assume that 'training_columns' was saved during training.
with open('training_columns.pkl', 'rb') as f:
    training_columns = pickle.load(f)

# Reindex the test features DataFrame; missing columns will be filled with zeros.
features_test = features_test.reindex(columns=training_columns, fill_value=0)

# Scale the test features using the saved scaler
X_test_scaled = scaler.transform(features_test)

# ===============================
# Load the saved models
# ===============================
# Load best Random Forest model
with open('best_rf_model.pkl', 'rb') as f:
    best_rf_model = pickle.load(f)

# Load best Gradient Boosting model
with open('best_gb_model.pkl', 'rb') as f:
    best_gb_model = pickle.load(f)

# ===============================
# Evaluate Models on Test Data
# ===============================
# Evaluate Random Forest model
start_time = time.time()
rf_predictions = best_rf_model.predict(X_test_scaled)
rf_runtime = (time.time() - start_time) / len(X_test_scaled)  # Average prediction time per sample

rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
rf_mse = mean_squared_error(y_test, rf_predictions)

# Evaluate Gradient Boosting model
start_time = time.time()
gb_predictions = best_gb_model.predict(X_test_scaled)
gb_runtime = (time.time() - start_time) / len(X_test_scaled)

gb_mae = mean_absolute_error(y_test, gb_predictions)
gb_rmse = np.sqrt(mean_squared_error(y_test, gb_predictions))
gb_mse = mean_squared_error(y_test, gb_predictions)

# Print evaluation metrics
print("Final Evaluation Metrics:")
print("Random Forest - MAE: {:.4f}, RMSE: {:.4f}, MSE: {:.4f}, Avg Runtime per sample: {:.6f} sec".format(
    rf_mae, rf_rmse, rf_mse, rf_runtime))
print("Gradient Boosting - MAE: {:.4f}, RMSE: {:.4f}, MSE: {:.4f}, Avg Runtime per sample: {:.6f} sec".format(
    gb_mae, gb_rmse, gb_mse, gb_runtime))

# Compare with baseline force_pre (if available)
# Calculate baseline evaluation metrics using force_pre column
baseline_mae = mean_absolute_error(y_test, baseline_force_pre)
baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_force_pre))
baseline_mse = mean_squared_error(y_test, baseline_force_pre)

print("Baseline (force_pre) - MAE: {:.4f}, RMSE: {:.4f}, MSE: {:.4f}".format(
    baseline_mae, baseline_rmse, baseline_mse))


Final Evaluation Metrics:
Random Forest - MAE: 5055189.8127, RMSE: 6575285.0083, MSE: 43234372940812.3203, Avg Runtime per sample: 0.000022 sec
Gradient Boosting - MAE: 4952478.7807, RMSE: 6351928.0018, MSE: 40346989340435.9062, Avg Runtime per sample: 0.000003 sec
Baseline (force_pre) - MAE: 2898684.1133, RMSE: 3775600.3207, MSE: 14255157781834.8516


In [ ]:
# Install Keras Tuner if not already installed
!pip install keras-tuner

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import keras_tuner as kt
import numpy as np

# Define a model-building function for the tuner
def build_model(hp):
    """
    Build a DNN model with hyperparameters to tune.
    """
    model = keras.Sequential()

    # Input layer using an Input layer is preferred
    model.add(keras.Input(shape=(X_train.shape[1],)))  # X_train为训练集特征，确保在运行前定义好

    # Tune the number of layers (at least 1 layer)
    num_layers = hp.Int('num_layers', 1, 3, default=2)

    for i in range(num_layers):
        # Tune the number of units in this layer
        units = hp.Int(f'units_{i}', min_value=32, max_value=256, step=32, default=64)
        # Tune dropout rate for this layer
        dropout_rate = hp.Float(f'dropout_{i}', 0.0, 0.5, step=0.1, default=0.2)
        # Tune L2 regularization factor
        l2_reg = hp.Float(f'l2_reg_{i}', 1e-5, 1e-3, sampling='LOG', default=1e-4)

        model.add(layers.Dense(units, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))

    # Output layer
    model.add(layers.Dense(1))

    # Tune learning rate for the optimizer
    learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG', default=1e-3)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mse',
                  metrics=['mae'])
    return model

# Set up the tuner: Here we use Hyperband search algorithm
tuner = kt.Hyperband(
    build_model,
    objective='val_mae',
    max_epochs=50,
    factor=3,
    directory='dnn_tuner_dir',
    project_name='force_prediction_tuning'
)

# Optional: Early stopping callback to stop training if no improvement
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Perform hyperparameter search
tuner.search(X_train, y_train,
             epochs=50,
             validation_split=0.2,
             callbacks=[stop_early],
             verbose=1)

# Retrieve the best hyperparameters and build the best model
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters found:", best_hp.values)

# Build the best model and train it
best_model = tuner.hypermodel.build(best_hp)
history = best_model.fit(X_train, y_train,
                         epochs=50,
                         validation_split=0.2,
                         callbacks=[stop_early],
                         verbose=1)

# 保存最佳模型及相关Scaler等（确保之前训练时保存了Scaler和训练列）
best_model.save('optimized_dnn_model.h5')
print("Optimized DNN model has been saved successfully.")

# 若需要后续加载最佳模型，请使用以下代码：
# loaded_best_model = tf.keras.models.load_model('optimized_dnn_model.h5', compile=False)


Trial 89 Complete [00h 01m 10s]
val_mae: 50453676.0

Best val_mae So Far: 47161284.0
Total elapsed time: 00h 29m 07s

Search: Running Trial #90

Value             |Best Value So Far |Hyperparameter
2                 |3                 |num_layers
192               |64                |units_0
0.3               |0.3               |dropout_0
0.00023201        |1.1315e-05        |l2_reg_0
256               |224               |units_1
0.4               |0.2               |dropout_1
0.00051684        |4.1139e-05        |l2_reg_1
0.00065826        |0.0067641         |learning_rate
256               |256               |units_2
0.1               |0.2               |dropout_2
2.4462e-05        |2.015e-05         |l2_reg_2
50                |50                |tuner/epochs
0                 |17                |tuner/initial_epoch
0                 |3                 |tuner/bracket
0                 |3                 |tuner/round

Epoch 1/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 270320